# Testing BorutaPy

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.dataset.data import Dataset
from src.features.build_features import *
from src.model.train import *

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

import matplotlib.pyplot as plt

pd.options.display.max_columns = None

%matplotlib inline

In [3]:
ds = Dataset()
ds.load_dataset()
build_processed_dataset(ds)

c:\workspaceperso\kaggle\ieee-fraud-detection\src\features\build_features.py:579: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if nan_constant in lbl.classes_:


In [4]:
X_train = ds.X_train
y_train = ds.y_train

In [5]:
X_train = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X_train)

In [8]:
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
X_train = X_train
y_train = y_train.values.ravel()

In [13]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=2, class_weight='balanced', max_depth=5)

In [14]:
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

In [15]:
# find all relevant features
feat_selector.fit(X_train, y_train)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	443
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	443
Rejected: 	0


KeyboardInterrupt: 

In [ ]:
# check selected features
feat_selector.support_

In [ ]:
# check ranking of features
feat_selector.ranking_